In [11]:
from pathlib import Path
from glob import glob

import pandas as pd
import numpy as np

from ase import io, Atoms, Atom
from pmutt.statmech import StatMech, trans, vib, rot, elec

# interactive plots
%matplotlib widget

## Sorted according SO-ZORA

In [21]:
working_directory = Path("/home/danian/hg2w/calculaT/ThermoChemistry/structures")

In [23]:
list_file_adf = glob(str(working_directory / 'sozora') + '/*.out')
energies_file_adf = working_directory / ('sozora/energies.txt')

## Reading Coordinates and Frequencies from ADF.2013

In [24]:
find_lines_adf = {}
for file_adf in list_file_adf:
    read_file_adf = Path(file_adf).read_text()

    """ Lines to read the geometry in ADF.2013 """
    start_line_geom = None
    last_line_geom = None
    start_line_freq = None
    last_line_freq = None
    for count, line in enumerate(read_file_adf.split('\n')):

        if line == "                                            *  F R E Q U E N C I E S  *":
            start_line_geom = count + 11
        elif line == " Atomic Masses:":
            last_line_geom = count - 4
        elif line == " List of All Frequencies:":
            start_line_freq = count + 9
        elif line == " Statistical Thermal Analysis  ***  ideal gas assumed  ***":
            last_line_freq = count - 4
        elif start_line_geom and last_line_geom and start_line_freq and last_line_freq:

            break
    find_lines_adf[Path(file_adf).name] = [start_line_geom, last_line_geom, start_line_freq, last_line_freq]


In [25]:
geoms_systems = {}
freqs_systems = {}
for file_adf in list_file_adf:
    with open(Path(file_adf), 'r') as f_adf:
        file_content = f_adf.readlines()

        """Atoms object with ase.Atoms"""
        last_line_geom = find_lines_adf[Path(file_adf).name][1]
        start_line_geom = find_lines_adf[Path(file_adf).name][0]
        lines_geom = last_line_geom - start_line_geom
        coord_system = []
        for i in range(lines_geom):
            line = file_content[start_line_geom + i].split()
            coord_system.append(Atom(line[1], (line[5], line[6], line[7])))
        geoms_systems[Path(file_adf).name] = Atoms(coord_system)

        """Frequencies"""
        last_line_freq = find_lines_adf[Path(file_adf).name][3]
        start_line_freq = find_lines_adf[Path(file_adf).name][2]
        lines_freq = last_line_freq - start_line_freq
        freq_system = []
        for i in range(lines_freq):
            line = file_content[start_line_freq + i].split()
            if float(line[0]) > 20.0:
                freq_system.append(float(line[0]))
        freqs_systems[Path(file_adf).name] = freq_system

In [ ]:
translation = {}
vibration = {}
rotation = {}
electronic = {}
energies = open(energies_file_adf, 'r').readlines()
for count, file_adf in enumerate(list_file_adf):
    name_file = Path(file_adf).name

    '''Translational'''
    translation[name_file] = trans.FreeTrans(n_degrees=3, atoms=geoms_systems[name_file])

    '''Vibrational'''
    vibration[name_file] = vib.HarmonicVib(vib_wavenumbers=freqs_systems[name_file]) #cm^-1

    '''Rotational'''
    rotation[name_file] = rot.RigidRotor(symmetrynumber=1, atoms=geoms_systems[name_file]) #simmetry point C1

    '''Electronic'''
    electronic[name_file] = elec.GroundStateElec(potentialenergy=energies[count], spin=0) #Ev

energies.close()

In [ ]:
statmech = {}
for file_adf in list_file_adf:
    name_file = Path(file_adf).name

    '''StatMech Initialization'''
    statmech[name_file] = StatMech(name= name_file.with_suffix(''),
                            trans_model=translation[name_file],
                            vib_model=vibration[name_file],
                            rot_model=rotation[name_file],
                            elec_model=electronic[name_file])

In [ ]:
print('systems   H[kcal/mol]    S[kcal/mol/K]    G[kcal/mol]     T: 298.15 K')
for file_adf in list_file_adf:
    name_file = Path(file_adf).name
    H_statmech = statmech[name_file].get_H(T=298.15, units='kcal/mol')
    S_statmech = statmech[name_file].get_S(T=298.15, units='kcal/mol/K')
    G_statmech = statmech[name_file].get_G(T=298.15, units='kcal/mol')
    print(f"{name_file.with_suffix('')}, {H_statmech:.3f}, {S_statmech}:.3f, {G_statmech}:.3f")
    # print(f'H_w6s1(T=298.15) = {H_statmech:.3f} kcal/mol')
    # print(f'S_w6s1(T=298.15) = {S_statmech:.3f} kcal/mol/K')
    # print(f'G_w6s1(T=298.15) = {G_statmech:.3f} kcal/mol')